In [66]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import hvplot.pandas
import requests
import json
import calendar
%matplotlib inline

In [67]:
sar_data_2020= Path("..\FInCen CSV Data\FinCen SAR Stats 2020 (Months).csv")
sar_2020_df= pd.read_csv(sar_data_2020, thousands=',', index_col='Year Month')
sar_2020_df

,State,Industry,Suspicious Activity,Count
Year Month,,,,
2020 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Check,2
2020 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Credit/Debit Card,2
2020 January,Alabama,Casino/Card Club - Tribal Authorized Casino,[Total],4
2020 January,Alabama,Depository Institution,ACH,96
2020 January,Alabama,Depository Institution,Advance Fee,4
...,...,...,...,...
2020 December,Wyoming,Money Services Business (MSB),[Total],28
2020 December,Wyoming,[Total],[Total],106
2020 December,[Total],[Total],[Total],123722


In [68]:
sar_2020_df=sar_2020_df.loc[sar_2020_df["Industry"]!="[Total]" ]
sar_2020_df=sar_2020_df.loc[sar_2020_df["Suspicious Activity"]!="[Total]" ]
sar_2020_df=sar_2020_df.loc[sar_2020_df["State"]!="[Total]" ]
sar_2020_df.loc[sar_2020_df.index.str.contains("[Total]") ]
sar_2020_df

,State,Industry,Suspicious Activity,Count
Year Month,,,,
2020 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Check,2
2020 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Credit/Debit Card,2
2020 January,Alabama,Depository Institution,ACH,96
2020 January,Alabama,Depository Institution,Advance Fee,4
2020 January,Alabama,Depository Institution,Business Loan,2
...,...,...,...,...
2020 December,Wyoming,Depository Institution,Other Fraud (Type),13
2020 December,Wyoming,Depository Institution,Wire,14
2020 December,Wyoming,Insurance Company,Wire,1


In [69]:
series_list1 = {
'LAUST010000000000003':'Alabama', 'LAUST020000000000003':'Alaska', 'LAUST040000000000003':'Arizona', 'LAUST050000000000003':'Arkansas', 'LAUST060000000000003':'California', 
'LAUST080000000000003':'Colorado', 'LAUST090000000000003':'Connecticut', 'LAUST100000000000003':'Delaware', 'LAUST120000000000003':'Florida', 'LAUST130000000000003':'Georgia', 
'LAUST150000000000003':'Hawaii', 'LAUST160000000000003':'Idaho', 'LAUST170000000000003':'Illinois', 'LAUST180000000000003':'Indiana', 
'LAUST190000000000003':'Iowa', 'LAUST200000000000003':'Kansas', 'LAUST210000000000003':'Kentucky', 'LAUST220000000000003':'Louisiana', 'LAUST230000000000003':'Maine', 
'LAUST240000000000003':'Maryland', 'LAUST250000000000003':'Massachusetts', 'LAUST260000000000003':'Michigan', 'LAUST270000000000003':'Minnesota', 'LAUST280000000000003':'Mississippi',
'LAUST290000000000003':'Missouri'} 
series_list2 = {
'LAUST300000000000003':'Montana', 'LAUST310000000000003':'Nebraska', 'LAUST320000000000003':'Nevada', 'LAUST330000000000003':'New Hampshire', 'LAUST340000000000003':'New Jersey', 
'LAUST350000000000003':'New Mexico', 'LAUST360000000000003':'New York', 'LAUST370000000000003':'North Carolina', 'LAUST380000000000003':'North Dakota', 'LAUST390000000000003':'Ohio', 'LAUST400000000000003':'Oklahoma', 
'LAUST410000000000003':'Oregon', 'LAUST420000000000003':'Pennsylvania', 'LAUST440000000000003':'Rhode Island', 'LAUST450000000000003':'South Carolina', 'LAUST460000000000003':'South Dakota', 'LAUST470000000000003':'Tennessee', 
'LAUST480000000000003':'Texas', 'LAUST490000000000003':'Utah', 'LAUST500000000000003':'Vermont', 'LAUST510000000000003':'Virginia', 
'LAUST530000000000003':'Washington', 'LAUST540000000000003':'West Virginia', 'LAUST550000000000003':'Wisconsin', 'LAUST560000000000003':'Wyoming',
}
df = pd.DataFrame(columns= ["State","Year","Month","Unemployment Rate"])

In [70]:
headers = {'Content-type': 'application/json'}
data1 = json.dumps({"seriesid": list(series_list1.keys()),"startyear":"2019", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data1, headers=headers)
json_data1 = json.loads(p.text)
data2 = json.dumps({"seriesid": list(series_list2.keys()),"startyear":"2019", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data2, headers=headers)
json_data2 = json.loads(p.text)

for series in json_data1['Results']['series']:
    state = series_list1[series['seriesID']]
    for item in series['data']:
        year = item['year']
        month = calendar.month_name[int(item['period'][1:3])]
        value = item['value']
        df = df.append({'State':state,'Year':year,'Month':month,'Unemployment Rate':value},ignore_index=True)

for series in json_data2['Results']['series']:
    state = series_list2[series['seriesID']]
    for item in series['data']:
        year = item['year']
        month = calendar.month_name[int(item['period'][1:3])]
        value = item['value']
        df = df.append({'State':state,'Year':year,'Month':month,'Unemployment Rate':value},ignore_index=True)
df

,State,Year,Month,Unemployment Rate
0,Alabama,2021,December,2.6
1,Alabama,2021,November,2.6
2,Alabama,2021,October,2.9
3,Alabama,2021,September,3.0
4,Alabama,2021,August,3.7
...,...,...,...,...
1795,Wyoming,2019,May,3.3
1796,Wyoming,2019,April,3.4
1797,Wyoming,2019,March,4.0
1798,Wyoming,2019,February,3.9


In [71]:
df_2020 = df.loc[df['Year']== '2020'].copy()
df_2020['Year Month'] = df_2020['Year'] +' '+ df_2020['Month']
df_2020 = df_2020[['Year Month', 'State','Unemployment Rate']]
df_2020.set_index('Year Month')

,State,Unemployment Rate
Year Month,,
2020 December,Alabama,4.1
2020 November,Alabama,4.4
2020 October,Alabama,4.9
2020 September,Alabama,6.0
2020 August,Alabama,6.7
...,...,...
2020 May,Wyoming,8.8
2020 April,Wyoming,5.3
2020 March,Wyoming,6.6


In [72]:
grouped_stats = sar_2020_df[["State", "Count"]].copy()
grouped_stats = grouped_stats.groupby('State').count()
grouped_stats

,Count
State,
APO / DPO / FPO,13
Alabama,271
Alaska,137
American Samoa,10
Arizona,354
...,...
Virginia,373
Washington,389
West Virginia,147


In [73]:
mapped_data = grouped_stats.hvplot.bar(
    rot = 90,
    frame_width = 700, frame_height = 500,
    label = "FinCen Fraud SAR Reporting by State in 2019"
)

mapped_data

:Bars   [State]   (Count)

In [74]:
sar_2020_df= sar_2020_df.reset_index()
sar_2020_df

,Year Month,State,Industry,Suspicious Activity,Count
0,2020 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Check,2
1,2020 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Credit/Debit Card,2
2,2020 January,Alabama,Depository Institution,ACH,96
3,2020 January,Alabama,Depository Institution,Advance Fee,4
4,2020 January,Alabama,Depository Institution,Business Loan,2
...,...,...,...,...,...
16338,2020 December,Wyoming,Depository Institution,Other Fraud (Type),13
16339,2020 December,Wyoming,Depository Institution,Wire,14
16340,2020 December,Wyoming,Insurance Company,Wire,1
16341,2020 December,Wyoming,Money Services Business (MSB),Other Fraud (Type),17


In [75]:
sar_2020_df= sar_2020_df.astype({'Count':'int'})
#sar_2020_df = sar_2020_df[['State', 'Count']]
#sar_2020_df = sar_2020_df.groupby(['Industry', 'State']).sum()
sar_by_state= sar_2020_df.hvplot.bar(hover_cols = 'Industry', hover=True, color = 'blue', x = 'Year Month', y = 'Count',groupby = 'State', rot=90,frame_width = 1000, frame_height = 500, label = 'FinCen Fraud SAR Reporting by State in 2021')
sar_by_state

:DynamicMap   [State]
   :Bars   [Year Month]   (Count,Industry)

In [76]:
industry_by_ftype = sar_2020_df.groupby(['Industry', 'Suspicious Activity']).sum()
industry_by_ftype.hvplot.bar(stacked=True, frame_height=500, width=1050, rot=90, legend='top_left', yformatter='%.0f')

:Bars   [Industry,Suspicious Activity]   (Count)

In [93]:
sar_grouped_stats= sar_2020_df[['State', 'Count', 'Year Month']]
sar_grouped_stats= sar_grouped_stats.groupby(['Year Month', 'State']).count()
sar_grouped_stats

Count
Year Month     State               
2020 April     Alabama           22
               Alaska            11
               Arizona           28
               Arkansas          20
               California        53
...                             ...
2020 September Virginia          30
               Washington        32
               West Virginia     12
               Wisconsin         25
               Wyoming           10

[696 rows x 1 columns]

In [98]:
#df_2020 = df_2020.reset_index()
#sar_grouped_stats= sar_grouped_stats.reset_index()
#df_2020= df_2020.sort_values('Year Month')

combined_df= pd.concat([df_2020, sar_grouped_stats], axis='columns', join='inner')
combined_df

,level_0,index,index,Year Month,State,Count
0,0,416,0,2020 April,Alabama,22
1,1,80,1,2020 April,Alaska,11
2,2,176,2,2020 April,Arizona,28
3,3,332,3,2020 April,Arkansas,20
4,4,500,4,2020 April,California,53
...,...,...,...,...,...,...
595,595,15,595,2020 October,Indiana,33
596,596,555,596,2020 October,Iowa,26
597,597,75,597,2020 October,Kansas,18
598,598,339,598,2020 October,Kentucky,27
